# the_dogs_paw

In [1]:
!pip install pandas
!pip install pystan==2.19.1.1
!pip install prophet
!pip install boto3

import pandas as pd
import numpy as np
from prophet import Prophet
import boto3
import os

  Using cached pystan-2.19.1.1.tar.gz (16.2 MB)
  Preparing metadata (setup.py) ... done
  Using cached cython-3.1.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.5 kB)
Using cached cython-3.1.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.2 MB)
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [3 lines of output]
      /tmp/pip-install-428dkp3e/pystan_3e6fa25fa947409fb6b9a0e0795e89a6/setup.py:61: DeprecationWarning: Attribute s is deprecated and will be removed in Python 3.14; use value instead
        self.version = node.value.s
      Cython>=0.22 and NumPy are required.
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pystan
  Running setup.py clean for pystan
Failed to build pystan
ERROR: Failed to build installable wheels for some pyproject.toml based projects (pystan)


In [2]:
import sagemaker
print(sagemaker.get_execution_role())


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
arn:aws:iam::059670545459:role/service-role/AmazonSageMaker-ExecutionRole-20250521T113889


## S3 Setup

In [3]:
bucket = 'thedogspaw-small-forecast-data'  # <--- change to your S3 bucket

# Connect to S3
s3 = boto3.client('s3')

def s3_get(key, local):
    s3.download_file(bucket, key, local)

# Download all required files to /tmp
s3_get('datasets/thedogspaw_phppos_items.csv',         '/tmp/items.csv')
s3_get('datasets/thedogspaw_phppos_locations.csv',     '/tmp/locations.csv')
s3_get('datasets/thedogspaw_phppos_location_items.csv','/tmp/location_items.csv')
s3_get('datasets/thedogspaw_phppos_sales.csv',         '/tmp/sales.csv')
s3_get('datasets/thedogspaw_phppos_sales_items.csv',   '/tmp/sales_items.csv')


In [4]:
items_df         = pd.read_csv('/tmp/items.csv')
locations_df     = pd.read_csv('/tmp/locations.csv')
locations_item_df= pd.read_csv('/tmp/location_items.csv')
sales_df         = pd.read_csv('/tmp/sales.csv', parse_dates=['sale_time'])
sales_items_df   = pd.read_csv('/tmp/sales_items.csv')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)   # set to None to show all rows if needed

display(items_df.head())


/tmp/ipykernel_17858/3107525308.py:1: DtypeWarning: Columns (56,74) have mixed types. Specify dtype option on import or set low_memory=False.
  items_df         = pd.read_csv('/tmp/items.csv')
/tmp/ipykernel_17858/3107525308.py:4: DtypeWarning: Columns (5,29,37,38,40,41,53,54,63) have mixed types. Specify dtype option on import or set low_memory=False.
  sales_df         = pd.read_csv('/tmp/sales.csv', parse_dates=['sale_time'])
/tmp/ipykernel_17858/3107525308.py:5: DtypeWarning: Columns (6,23) have mixed types. Specify dtype option on import or set low_memory=False.
  sales_items_df   = pd.read_csv('/tmp/sales_items.csv')


,name,category_id,supplier_id,manufacturer_id,item_number,product_id,ecommerce_product_id,ecommerce_product_quantity,description,size,tax_included,cost_price,unit_price,promo_price,start_date,end_date,reorder_level,expire_days,item_id,allow_alt_description,is_serialized,override_default_tax,is_ecommerce,is_service,is_ebt_item,commission_percent,commission_percent_type,commission_fixed,change_cost_price,disable_loyalty,deleted,last_modified,ecommerce_last_modified,tax_class_id,replenish_level,system_item,max_discount_percent,max_edit_price,min_edit_price,custom_field_1_value,custom_field_2_value,custom_field_3_value,custom_field_4_value,custom_field_5_value,custom_field_6_value,custom_field_7_value,custom_field_8_value,custom_field_9_value,custom_field_10_value,required_age,verify_age,weight,length,width,height,ecommerce_shipping_class_id,long_description,allow_price_override_regardless_of_permissions,main_image_id,only_integer,is_series_package,series_quantity,series_days_to_use_within,is_barcoded,default_quantity,disable_from_price_rules,last_edited,info_popup,item_inactive,barcode_name,tags,is_favorite,loyalty_multiplier,ecommerce_inventory_item_id,weight_unit,is_recurring,startup_cost,prorated,interval,weekday,day_number,month,day,shopify_item_level_inventory_policy,non_profit_sale,ecommerce_first_variation_id,limited_sales_quantity
0,Earthborn Holistic Grain Free - Meadow Feast -...,7.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,11.99,16.99,NaN,NaN,NaN,1.0,NaN,1,0,0,0,1,0,0,0.0,selling_price,0.0,0,0,1,2017-04-19 13:24:29,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,1,NaN,0,2017-04-19 13:24:29,NaN,0,NaN,NaN,0,NaN,NaN,NaN,0,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN
1,Puppy Groom - 25,13.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,12.50,25.00,NaN,NaN,NaN,1.0,NaN,2,1,0,0,1,1,0,NaN,NaN,NaN,0,0,1,2020-09-21 09:52:32,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,1,NaN,0,2017-04-19 13:24:29,NaN,0,NaN,NaN,0,NaN,NaN,NaN,0,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN
2,Puppy Groom - 30,13.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,15.00,30.00,NaN,NaN,NaN,1.0,NaN,3,1,0,0,1,1,0,NaN,NaN,NaN,0,0,1,2020-09-21 09:52:32,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,1,NaN,0,2018-09-28 15:41:48,NaN,0,NaN,NaN,0,NaN,NaN,NaN,0,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN
3,Puppy Groom - 35,13.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,17.50,35.00,NaN,NaN,NaN,1.0,NaN,4,1,0,0,1,1,0,NaN,NaN,NaN,0,0,1,2020-09-21 09:52:32,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,1,NaN,0,2017-04-19 13:24:29,NaN,0,NaN,NaN,0,NaN,NaN,NaN,0,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN
4,Bubble Bath - 35,13.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,17.50,35.00,NaN,NaN,NaN,1.0,NaN,5,1,0,0,1,1,0,NaN,NaN,NaN,0,0,1,2020-09-19 15:58:05,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,1,NaN,0,2019-07-09 15:28:39,NaN,0,NaN,NaN,0,NaN,NaN,NaN,0,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN


## Preprocess sales.csv, items.csv and sale_items.csv NaN columns to Str


In [5]:
sales_cols_as_str = [5, 29, 37, 38, 40, 41, 53, 54, 63]  # zero-based columns

sales_df = pd.read_csv(
    '/tmp/sales.csv',
    parse_dates=['sale_time'],
    dtype={col: str for col in sales_cols_as_str},
    low_memory=False
)
items_df = pd.read_csv(
    '/tmp/items.csv',
    dtype={56: str, 74: str},  # or use actual column names
    low_memory=False
)
sales_items_df = pd.read_csv(
    '/tmp/sales_items.csv',
    dtype={6: str, 23: str},   # or use names!
    low_memory=False
)

## Sanity Check

In [6]:
# Check for duplicate item_ids
duplicates = items_df[items_df.duplicated('item_id', keep=False)]
print(f"Found {duplicates['item_id'].nunique()} duplicated item_ids.")
print(duplicates.sort_values('item_id'))

# Check for duplicate item names
name_dupes = items_df[items_df.duplicated('name', keep=False)]
print(f"Found {name_dupes['name'].nunique()} duplicated item names.")
print(name_dupes.sort_values('name'))

# See name duplicate counts
name_counts = items_df['name'].value_counts()
dupe_names = name_counts[name_counts > 1].index.tolist()
print(f"Found {len(dupe_names)} item names with duplicates.")
print(dupe_names[:20])

# See all items with duplicate names
dupes = items_df[items_df['name'].isin(dupe_names)].sort_values('name')
print(dupes[['item_id', 'name', 'category_id', 'supplier_id']].head(30))


Found 0 duplicated item_ids.
Empty DataFrame
Columns: [name, category_id, supplier_id, manufacturer_id, item_number, product_id, ecommerce_product_id, ecommerce_product_quantity, description, size, tax_included, cost_price, unit_price, promo_price, start_date, end_date, reorder_level, expire_days, item_id, allow_alt_description, is_serialized, override_default_tax, is_ecommerce, is_service, is_ebt_item, commission_percent, commission_percent_type, commission_fixed, change_cost_price, disable_loyalty, deleted, last_modified, ecommerce_last_modified, tax_class_id, replenish_level, system_item, max_discount_percent, max_edit_price, min_edit_price, custom_field_1_value, custom_field_2_value, custom_field_3_value, custom_field_4_value, custom_field_5_value, custom_field_6_value, custom_field_7_value, custom_field_8_value, custom_field_9_value, custom_field_10_value, required_age, verify_age, weight, length, width, height, ecommerce_shipping_class_id, long_description, allow_price_override_r

## Merge Sales Items with Sales (Add date/store info to item sales) 

In [7]:
item_sales = sales_items_df.merge(
    sales_df[['sale_id', 'sale_time', 'location_id']],
    on='sale_id', how='left'
)
item_sales['date'] = pd.to_datetime(item_sales['sale_time']).dt.date
print(item_sales[['sale_id', 'item_id', 'location_id', 'date', 'quantity_purchased']].tail(10))


        sale_id  item_id  location_id        date  quantity_purchased
525719   204502     8185            1  2025-05-23                 1.0
525720   204502     8244            1  2025-05-23                 1.0
525721   204502     8651            1  2025-05-23                 1.0
525722   204502     9578            1  2025-05-23                 3.0
525723   204502    10874            1  2025-05-23                 4.0
525724   204502    11110            1  2025-05-23                 1.0
525725   204503    11543            1  2025-05-23                 1.0
525726   204504     8244            1  2025-05-23                 1.0
525727   204504     8247            1  2025-05-23                 2.0
525728   204504     8247            1  2025-05-23                 4.0


## Aggregate Daily Sales Per Item Per Store

In [10]:
recent_daily_item_sales = (
    item_sales.groupby(['location_id', 'item_id', 'date'])['quantity_purchased']
    .sum()
    .reset_index()
    .sort_values(['location_id', 'item_id', 'date'])
)
print(daily_item_sales.tail(10))

        location_id  item_id        date  quantity_purchased
344887            2    11522  2025-04-09                 1.0
344888            2    11595  2025-04-06                 1.0
344889            2    11595  2025-04-13                 1.0
344890            2    11595  2025-05-15                 1.0
344891            2    11600  2025-03-30                 1.0
344892            2    11622  2025-04-13                -1.0
344893            2    11658  2025-04-29                 1.0
344894            2    11658  2025-05-09                 1.0
344895            2    11660  2025-04-24                 1.0
344896            2    11727  2025-05-20                 1.0


## Calculate enough history (have a minimum 20 total sales day after filtering) for forecasting

In [11]:
item_day_counts = (
    recent_daily_item_sales.groupby(['location_id', 'item_id'])['date']
    .nunique()
    .reset_index()
    .rename(columns={'date': 'num_days_with_sales'})
)
item_day_counts['enough_history'] = item_day_counts['num_days_with_sales'] >= 20
print(item_day_counts.head(10))
print(f"Forecastable items: {item_day_counts['enough_history'].sum()} / {len(item_day_counts)}")


   location_id  item_id  num_days_with_sales  enough_history
0            1        1                    1           False
1            1        2                   13           False
2            1        3                   10           False
3            1        4                   45            True
4            1        5                  966            True
5            1        6                  841            True
6            1        7                  640            True
7            1        8                  622            True
8            1        9                  207            True
9            1       10                  413            True
Forecastable items: 2510 / 11467


## The main shit. Forecast / fallback per item per store

In [12]:
from tqdm.notebook import tqdm

lead_time_days = 7
z = 1.65
results = []

for _, row in tqdm(item_day_counts.iterrows(), total=len(item_day_counts)):
    loc = row['location_id']
    item = row['item_id']
    enough = row['enough_history']

    item_sales_history = daily_item_sales[
        (daily_item_sales['location_id'] == loc) &
        (daily_item_sales['item_id'] == item)
    ].copy()
    item_sales_history = item_sales_history.rename(columns={'date': 'ds', 'quantity_purchased': 'y'})
    item_sales_history['ds'] = pd.to_datetime(item_sales_history['ds'])

    # Apply 12-month cutoff in the loop, to match recent_daily_item_sales
    cutoff_in_loop = item_sales_history['ds'].max() - pd.DateOffset(months=12)
    item_sales_history = item_sales_history[item_sales_history['ds'] >= cutoff_in_loop]

    reorder_level = None
    replenish_level = None

    if enough and len(item_sales_history) >= 20:
        try:
            m = Prophet(daily_seasonality=True)
            m.fit(item_sales_history)
            future = m.make_future_dataframe(periods=lead_time_days)
            forecast = m.predict(future)
            lead_forecast = forecast.tail(lead_time_days)
            demand_lt = lead_forecast['yhat'].sum()
            sigma_lt = (lead_forecast['yhat_upper'].sum() - lead_forecast['yhat_lower'].sum()) / 3.29
            safety_stock = z * sigma_lt
            reorder_level = int(np.round(demand_lt + safety_stock))
            replenish_level = int(np.round(reorder_level + demand_lt))
        except Exception as e:
            last_week = item_sales_history.sort_values('ds').tail(7)
            avg_daily = last_week['y'].mean() if len(last_week) else 1
            demand_lt = avg_daily * lead_time_days
            reorder_level = int(np.round(demand_lt))
            replenish_level = int(np.round(demand_lt * 2))
    else:
        last_week = item_sales_history.sort_values('ds').tail(7)
        avg_daily = last_week['y'].mean() if len(last_week) else 1
        demand_lt = avg_daily * lead_time_days
        reorder_level = int(np.round(demand_lt))
        replenish_level = int(np.round(demand_lt * 2))

    results.append({
        'location_id': loc,
        'item_id': item,
        'reorder_level': reorder_level,
        'replenish_level': replenish_level,
        'enough_history': enough
    })

results_df = pd.DataFrame(results)
print(results_df.head(10))
print("-----------------")
print(results_df.tail(10))


  0%|          | 0/11467 [00:00<?, ?it/s]

08:03:12 - cmdstanpy - INFO - Chain [1] start processing
08:03:12 - cmdstanpy - INFO - Chain [1] done processing
08:03:12 - cmdstanpy - INFO - Chain [1] start processing
08:03:12 - cmdstanpy - INFO - Chain [1] done processing
08:03:12 - cmdstanpy - INFO - Chain [1] start processing
08:03:12 - cmdstanpy - INFO - Chain [1] done processing
08:03:13 - cmdstanpy - INFO - Chain [1] start processing
08:03:13 - cmdstanpy - INFO - Chain [1] done processing
08:03:13 - cmdstanpy - INFO - Chain [1] start processing
08:03:13 - cmdstanpy - INFO - Chain [1] done processing
08:03:13 - cmdstanpy - INFO - Chain [1] start processing
08:03:13 - cmdstanpy - INFO - Chain [1] done processing
08:03:13 - cmdstanpy - INFO - Chain [1] start processing
08:03:13 - cmdstanpy - INFO - Chain [1] done processing
08:03:13 - cmdstanpy - INFO - Chain [1] start processing
08:03:13 - cmdstanpy - INFO - Chain [1] done processing
08:03:13 - cmdstanpy - INFO - Chain [1] start processing
08:03:13 - cmdstanpy - INFO - Chain [1]

   location_id  item_id  reorder_level  replenish_level  enough_history
0            1        1             28               56           False
1            1        2              7               14           False
2            1        3              7               14           False
3            1        4              7               14            True
4            1        5             17               28            True
5            1        6             17               28            True
6            1        7             17               28            True
7            1        8             20               32            True
8            1        9             12               19            True
9            1       10             15               24            True
-----------------
       location_id  item_id  reorder_level  replenish_level  enough_history
11457            2    11501              7               14           False
11458            2    11502           

In [13]:
# Merge item names
merged_results = results_df.merge(items_df[['item_id', 'name']], on='item_id', how='left')

# Last sale date for each (location, item)
last_sale_dates = (
    recent_daily_item_sales.groupby(['location_id', 'item_id'])['date']
    .max()
    .reset_index()
    .rename(columns={'date': 'last_sale_date'})
)

merged_results = merged_results.merge(last_sale_dates, on=['location_id', 'item_id'], how='left')

print(merged_results[['location_id', 'item_id', 'name', 'reorder_level', 'replenish_level', 'last_sale_date']].head(20))


    location_id  item_id                                               name  \
0             1        1  Earthborn Holistic Grain Free - Meadow Feast -...   
1             1        2                                   Puppy Groom - 25   
2             1        3                                   Puppy Groom - 30   
3             1        4                                   Puppy Groom - 35   
4             1        5                                   Bubble Bath - 35   
5             1        6                                   Bubble Bath - 40   
6             1        7                                   Bubble Bath - 45   
7             1        8                                   Bubble Bath - 50   
8             1        9                                   Bubble Bath - 55   
9             1       10                                   Bubble Bath - 60   
10            1       11                                   Bubble Bath - 65   
11            1       12                            

In [14]:
results_key = 'results/thedogspaw_reorder_replenish_results.csv'
merged_results.to_csv('/tmp/thedogspaw_reorder_replenish_results.csv', index=False)
s3.upload_file('/tmp/thedogspaw_reorder_replenish_results.csv', bucket, results_key)
os.remove('/tmp/thedogspaw_reorder_replenish_results.csv')
print(f"✅ Reorder & replenish results written to s3://{bucket}/{results_key}")


✅ Reorder & replenish results written to s3://thedogspaw-small-forecast-data/results/thedogspaw_reorder_replenish_results.csv
